In [ ]:
import sys
sys.path.extend([ '../', '../../'])
import torch
import dataloader as dl
from args import Args
from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.pubmed()
torch.cuda.empty_cache()

In [ ]:
data = torch.load('../data/pubmed_data.pt')
print(data)

In [ ]:

print(data['d'].y[:10,:])
# args.device = torch.device("cpu")
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['g']['x'] = data['g']['x'].to(device)
data['d']['x'] = data['d']['x'].to(device)
data['c']['x'] = data['c']['x'].to(device)
data['s']['x'] = data['s']['x'].to(device)

# Send all y tensors to the device
data['d']['y'] = data['d']['y'].to(device)

data['g', 'walk', 'g']['edge_index'] = data['g', 'walk', 'g']['edge_index'].to(device) 
data['g', 'walk', 'd']['edge_index'] = data['g', 'walk', 'd']['edge_index'].to(device)
data['g', 'walk', 'c']['edge_index'] = data['g', 'walk', 'c']['edge_index'].to(device)
data['g', 'walk', 's']['edge_index'] = data['g', 'walk', 's']['edge_index'].to(device)

data['d', 'walk', 'g']['edge_index'] = data['d', 'walk', 'g']['edge_index'].to(device)
data['d', 'walk', 'd']['edge_index'] = data['d', 'walk', 'd']['edge_index'].to(device)
data['d', 'walk', 'c']['edge_index'] = data['d', 'walk', 'c']['edge_index'].to(device)
data['d', 'walk', 's']['edge_index'] = data['d', 'walk', 's']['edge_index'].to(device)

data['c', 'walk', 'g']['edge_index'] = data['c', 'walk', 'g']['edge_index'].to(device)
data['c', 'walk', 'd']['edge_index'] = data['c', 'walk', 'd']['edge_index'].to(device)
data['c', 'walk', 'c']['edge_index'] = data['c', 'walk', 'c']['edge_index'].to(device)
data['c', 'walk', 's']['edge_index'] = data['c', 'walk', 's']['edge_index'].to(device)

data['s', 'walk', 'g']['edge_index'] = data['s', 'walk', 'g']['edge_index'].to(device)
data['s', 'walk', 'd']['edge_index'] = data['s', 'walk', 'd']['edge_index'].to(device)
data['s', 'walk', 'c']['edge_index'] = data['s', 'walk', 'c']['edge_index'].to(device)
data['s', 'walk', 's']['edge_index'] = data['s', 'walk', 's']['edge_index'].to(device)

edge_indices = [ data['d', 'walk', 'g'].edge_index, data['d', 'walk', 'd'].edge_index, data['d', 'walk', 'c'].edge_index, data['d', 'walk', 's'].edge_index ]

In [ ]:
args.portion = 0
args.im_ratio = [0.7, 0.1, 0.2, 0.2, 0.4, 0.4]

In [ ]:
c_train_num = dl.train_num(data['d'].y[:,1], args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['d'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

In [ ]:
encoder1 = Het_ConEn(args.embed_dim, args.dropout)
encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# encoder1 = torch.load('pretrained/encoder1.pth')
# encoder2 = torch.load('pretrained/encoder2.pth')
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_g = EdgePredictor(args.embed_dim)
decoder_d = EdgePredictor(args.embed_dim)
decoder_c = EdgePredictor(args.embed_dim)
decoder_s = EdgePredictor(args.embed_dim)
# decoder_g = torch.load('pretrained/decoder_g.pth')
# decoder_d = torch.load('pretrained/decoder_d.pth')
# decoder_c = torch.load('pretrained/decoder_c.pth')
# decoder_s = torch.load('pretrained/decoder_s.pth')

decoder_list = [decoder_g, decoder_d, decoder_c, decoder_s]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [ ]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'smote', train_mode = '')

In [ ]:
# test_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

In [ ]:
# torch.save(decoder_list[0], 'pretrained/decoder_g.pth')
# torch.save(decoder_list[1], 'pretrained/decoder_d.pth')
# torch.save(decoder_list[2], 'pretrained/decoder_c.pth')
# torch.save(decoder_list[3], 'pretrained/decoder_s.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')